In [20]:
import requests

r = requests.get("https://gamma-api.polymarket.com/events?closed=false")
response = r.json()

response

[{'id': '16084',
  'ticker': 'fed-rate-hike-in-2025',
  'slug': 'fed-rate-hike-in-2025',
  'title': 'Fed rate hike in 2025?',
  'description': 'This market will resolve to “Yes” if the upper bound of the target federal funds rate is increased at any point between January 1, 2025 and the Fed\'s December 2025 meeting, currently scheduled for December 9-10. Otherwise, this market will resolve to “No”.\n\nThis market may not resolve to "No" until the Fed has released its rate changes information following its December meeting.\n\nThe primary resolution source for this market will be the official website of the Federal Reserve (https://www.federalreserve.gov/monetarypolicy/openmarket.htm), however a consensus of credible reporting may also be used.',
  'startDate': '2024-12-29T22:50:44.013518Z',
  'creationDate': '2024-12-29T22:50:44.013516Z',
  'endDate': '2025-12-10T12:00:00Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-the-fed-raise-interest-rates-in-2025-PQTEY

In [2]:
import requests

# Step 1: Pull all active and non-active Polymarket data with pagination
print("=== PULLING ALL POLYMARKET DATA ===")

def fetch_all_paginated_data(url, params=None):
    """Fetch all data from a paginated API endpoint"""
    all_data = []
    cursor = None
    
    while True:
        current_params = params.copy() if params else {}
        current_params['limit'] = 1000  # Max limit per request
        
        if cursor:
            current_params['cursor'] = cursor
            
        try:
            response = requests.get(url, params=current_params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            # Handle different response formats
            if isinstance(data, list):
                items = data
                next_cursor = None
            else:
                items = data.get('data', [])
                next_cursor = data.get('next') or data.get('nextCursor')
            
            all_data.extend(items)
            print(f"Fetched {len(items)} items (total: {len(all_data)})")
            
            if not next_cursor:
                break
            cursor = next_cursor
            
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    return all_data

# Fetch all events (active and closed)
print("Fetching all events...")
all_events = []

# Active events
print("Fetching active events...")
active_events = fetch_all_paginated_data("https://gamma-api.polymarket.com/events", {"closed": "false"})
all_events.extend(active_events)
print(f"Found {len(active_events)} active events")

# Closed events
print("Fetching closed events...")
closed_events = fetch_all_paginated_data("https://gamma-api.polymarket.com/events", {"closed": "true"})
all_events.extend(closed_events)
print(f"Found {len(closed_events)} closed events")

# Fetch all markets (active and closed)
print("Fetching all markets...")
all_markets = []

# Active markets
print("Fetching active markets...")
active_markets = fetch_all_paginated_data("https://gamma-api.polymarket.com/markets", {"active": "true"})
all_markets.extend(active_markets)
print(f"Found {len(active_markets)} active markets")

# Closed markets
print("Fetching closed markets...")
closed_markets = fetch_all_paginated_data("https://gamma-api.polymarket.com/markets", {"closed": "true"})
all_markets.extend(closed_markets)
print(f"Found {len(closed_markets)} closed markets")

print(f"\nTotal events: {len(all_events)}")
print(f"Total markets: {len(all_markets)}")

=== PULLING ALL POLYMARKET DATA ===
Fetching all events...
Fetching active events...
Fetched 500 items (total: 500)
Found 500 active events
Fetching closed events...
Fetched 500 items (total: 500)
Found 500 closed events
Fetching all markets...
Fetching active markets...
Fetched 500 items (total: 500)
Found 500 active markets
Fetching closed markets...
Fetched 500 items (total: 500)
Found 500 closed markets

Total events: 1000
Total markets: 1000


In [3]:
# Show all event titles
print("=== ALL EVENT TITLES ===")
for i, event in enumerate(all_events):
    if not isinstance(event, dict):
        continue
    
    title = event.get("title", event.get("name", event.get("question", "No title")))
    print(f"{i+1}. {title}")

=== ALL EVENT TITLES ===
1. Fed rate hike in 2025?
2. How many Fed rate cuts in 2025?
3. Which CEOs will be gone in 2025?
4. Largest Company end of 2025?
5. US recession in 2025?
6. Fed emergency rate cut in 2025?
7. Which company has best AI model end of 2025?
8. What price will Bitcoin hit in 2025?
9. What price will Ethereum hit in 2025?
10. Tether insolvent in 2025?
11. USDT depeg in 2025?
12. Highest grossing movie in 2025?
13. How many SpaceX launches in 2025?
14. Weed rescheduled in 2025?
15. Ukraine joins NATO in 2025?
16. Khamenei out as Supreme Leader of Iran in 2025?
17. Nuclear weapon detonation in 2025?
18. Russia x Ukraine ceasefire in 2025?
19. Putin out as President of Russia in 2025?
20. Netanyahu out by...?
21. One Direction reunion in 2025?
22. Britney Spears tour in 2025?
23. Nara & Lucky divorce in 2025?
24. Lana Del Rey divorce in 2025?
25. Hailey Bieber pregnant in 2025?
26. New pandemic in 2025?
27. Will the US confirm that aliens exist in 2025?
28. MicroStrateg

In [22]:
import re

# Expanded search for Fed-related events with debugging
print("=== EXPANDED SEARCH FOR FED-RELATED EVENTS (WITH DEBUG) ===\n")

fed_events = []
fed_event_titles = []

# Keywords to search for (in title OR description)
fed_keywords = [
    r'\bfed\b',                    # "fed" as word
    r'\bfomc\b',                   # FOMC meetings
    r'federal\s+reserve',          # Federal Reserve
    r'rate\s+decision',             # Rate decisions
    r'interest\s+rate',             # Interest rates
    r'bps\b',                       # Basis points
    r'25\s*bps|50\s*bps|75\s*bps', # Specific rate changes
    r'no\s+change',                 # No change decisions
    r'jerome\s+powell',             # Fed Chair
    r'december.*decision',          # December decisions
    r'decision.*december',         # Decision in December
]

# Also search for specific known titles
known_titles = [
    'fed decision',
    'fomc decision',
    'rate decision',
    'december decision',
]

for i, event in enumerate(all_events):
    if not isinstance(event, dict):
        continue
    
    title = event.get("title", event.get("name", event.get("question", "")))
    description = event.get("description", "")
    
    # Combine title and description for searching
    search_text = f"{title} {description}".lower()
    
    # Check if any keyword matches
    matched = False
    matched_keyword = None
    
    # First check keywords
    for keyword in fed_keywords:
        if re.search(keyword, search_text, re.IGNORECASE):
            matched = True
            matched_keyword = keyword
            break
    
    # Also check if title contains known patterns
    if not matched:
        for known_title in known_titles:
            if known_title in search_text:
                matched = True
                matched_keyword = f"known_title: {known_title}"
                break
    
    if matched:
        fed_event_titles.append({
            "index": i,
            "title": title,
            "event_id": event.get("id", "No ID"),
            "closed": event.get("closed", "Unknown"),
            "matched_keyword": matched_keyword
        })
        print(f"✓ {title}")
        print(f"  Matched: {matched_keyword}")
        fed_events.append(event)

print(f"\nFound {len(fed_event_titles)} Fed-related events")

# Also search for events with "decision" in title to see what we're missing
print("\n=== SEARCHING FOR ALL 'DECISION' EVENTS ===")
decision_events = []
for i, event in enumerate(all_events):
    if not isinstance(event, dict):
        continue
    title = event.get("title", event.get("name", event.get("question", "")))
    if 'decision' in title.lower():
        decision_events.append(title)
        print(f"  - {title}")

print(f"\nFound {len(decision_events)} events with 'decision' in title")

# Search for events with "december" in title
print("\n=== SEARCHING FOR ALL 'DECEMBER' EVENTS ===")
december_events = []
for i, event in enumerate(all_events):
    if not isinstance(event, dict):
        continue
    title = event.get("title", event.get("name", event.get("question", "")))
    if 'december' in title.lower():
        december_events.append(title)
        print(f"  - {title}")

print(f"\nFound {len(december_events)} events with 'december' in title")

=== EXPANDED SEARCH FOR FED-RELATED EVENTS (WITH DEBUG) ===

✓ Fed rate hike in 2025?
  Matched: \bfed\b
✓ How many Fed rate cuts in 2025?
  Matched: \bfed\b
✓ Fed emergency rate cut in 2025?
  Matched: \bfed\b
✓ US national Bitcoin reserve in 2025?
  Matched: federal\s+reserve
✓ US national Solana reserve in 2025?
  Matched: federal\s+reserve
✓ US national XRP reserve in 2025?
  Matched: federal\s+reserve
✓ US national Ethereum reserve in 2025?
  Matched: federal\s+reserve
✓ US national Dogecoin reserve in 2025?
  Matched: federal\s+reserve
✓ Jerome Powell out as Fed Chair in 2025?
  Matched: \bfed\b
✓ How much spending will DOGE cut in 2025?
  Matched: federal\s+reserve
✓ Will Elon cut the budget by at least 10% in 2025?
  Matched: federal\s+reserve
✓ Will Elon cut the budget by at least 5% in 2025?
  Matched: federal\s+reserve
✓ Fed abolished in 2025?
  Matched: \bfed\b
✓ Le Pen sentence reduced in 2025?
  Matched: decision.*december
✓ Major U.S. bank bailout in 2025?
  Matched: fed

In [4]:
import re

# Check for exact word "fed" (not as part of other words)
print("=== CHECKING FOR EXACT WORD 'FED' ===")
fed_events = []
fed_event_titles = []
for i, event in enumerate(all_events):
    if not isinstance(event, dict):
        continue
    
    title = event.get("title", event.get("name", event.get("question", "")))
    
    # Use word boundary regex to match only "fed" as a complete word
    if re.search(r'\bfed\b', title, re.IGNORECASE):
        fed_event_titles.append({
            "index": i,
            "title": title,
            "event_id": event.get("id", "No ID"),
            "closed": event.get("closed", "Unknown")
        })
        print(f"✓ {title}")
        fed_events.append(event)

print(f"\nFound {len(fed_event_titles)} events with exact word 'fed' in the title")

for event in fed_events:
    print(event)

=== CHECKING FOR EXACT WORD 'FED' ===
✓ Fed rate hike in 2025?
✓ How many Fed rate cuts in 2025?
✓ Fed emergency rate cut in 2025?
✓ Jerome Powell out as Fed Chair in 2025?
✓ Fed abolished in 2025?
✓ Who will Trump announce as next Fed Chair in 2025?

Found 6 events with exact word 'fed' in the title
{'id': '16084', 'ticker': 'fed-rate-hike-in-2025', 'slug': 'fed-rate-hike-in-2025', 'title': 'Fed rate hike in 2025?', 'description': 'This market will resolve to “Yes” if the upper bound of the target federal funds rate is increased at any point between January 1, 2025 and the Fed\'s December 2025 meeting, currently scheduled for December 9-10. Otherwise, this market will resolve to “No”.\n\nThis market may not resolve to "No" until the Fed has released its rate changes information following its December meeting.\n\nThe primary resolution source for this market will be the official website of the Federal Reserve (https://www.federalreserve.gov/monetarypolicy/openmarket.htm), however a con

In [ ]:

url = "https://clob.polymarket.com/prices-history"

querystring = {"market":"78970393675220813117457137074949547635027336958817818487077806376171274244367","interval":"max","fidelity":"60"}

response = requests.get(url, params=querystring)

print(response.json())

{'id': '538291', 'question': 'Will Trump announce Kevin Warsh as next Fed Chair in 2025?', 'conditionId': '0xf1f111ef03daf045896e26beb18882cf99b80f5ffb07b525ff8242d2bad47944', 'slug': 'will-trump-announce-kevin-warsh-as-next-fed-chair-in-2025', 'resolutionSource': '', 'endDate': '2025-12-31T12:00:00Z', 'liquidity': '5983.00405', 'startDate': '2025-04-18T18:00:35.37Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/who-will-trump-announce-as-next-fed-chair-7-caEJvKh7b8.jpg', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/who-will-trump-announce-as-next-fed-chair-7-caEJvKh7b8.jpg', 'description': 'Trump\'s Treasury Secretary pick Scott Bessent has floated the idea of Trump naming a successor to Jerome Powell to serve as a "shadow chair", prior to the end of Powell\'s term, scheduled for May 2026. (see: https://www.nbcwashington.com/news/business/money-report/fed-chair-jerome-powell-can-serve-remainder-of-term-says-trumps-treasury-pick-bessent/3788991/).\n\n

In [18]:
import json

for event in fed_events:
    if 'markets' in event and len(event['markets']) > 0:
        market = event['markets'][0]
        
        # Get clobTokenIds (it's a JSON string, so parse it)
        clob_token_ids_str = market.get('clobTokenIds', '[]')
        clob_token_ids = json.loads(clob_token_ids_str)
        
        if len(clob_token_ids) > 0:
            # Use the first token ID (usually for "Yes" outcome)
            market_id = clob_token_ids[0]
            
            # Now use this in your API call
            url = "https://clob.polymarket.com/prices-history"
            querystring = {"market": str(market_id), "interval": "max", "fidelity": "60"}
            response = requests.get(url, params=querystring)
            print(response.json())

{'history': [{'t': 1759773606, 'p': 0.026}, {'t': 1759777203, 'p': 0.026}, {'t': 1759780804, 'p': 0.026}, {'t': 1759784403, 'p': 0.026}, {'t': 1759788009, 'p': 0.026}, {'t': 1759791615, 'p': 0.026}, {'t': 1759795209, 'p': 0.026}, {'t': 1759798803, 'p': 0.026}, {'t': 1759802405, 'p': 0.026}, {'t': 1759806012, 'p': 0.026}, {'t': 1759809605, 'p': 0.026}, {'t': 1759813206, 'p': 0.026}, {'t': 1759816805, 'p': 0.026}, {'t': 1759820402, 'p': 0.026}, {'t': 1759824012, 'p': 0.022}, {'t': 1759827602, 'p': 0.0225}, {'t': 1759831213, 'p': 0.024}, {'t': 1759834810, 'p': 0.022}, {'t': 1759838408, 'p': 0.0235}, {'t': 1759842005, 'p': 0.022}, {'t': 1759845613, 'p': 0.0225}, {'t': 1759849202, 'p': 0.0235}, {'t': 1759852806, 'p': 0.0235}, {'t': 1759856405, 'p': 0.023}, {'t': 1759860017, 'p': 0.0215}, {'t': 1759863603, 'p': 0.0225}, {'t': 1759867206, 'p': 0.022}, {'t': 1759870807, 'p': 0.0215}, {'t': 1759874407, 'p': 0.023}, {'t': 1759878014, 'p': 0.0235}, {'t': 1759881613, 'p': 0.0235}, {'t': 1759885203

In [19]:
import pandas as pd
import requests
import json
from datetime import datetime

all_data = []

for event in fed_events:
    # Get event title
    event_title = event.get('title', 'Unknown')
    
    # Get market ID from clobTokenIds
    if 'markets' in event and len(event['markets']) > 0:
        market = event['markets'][0]
        clob_token_ids_str = market.get('clobTokenIds', '[]')
        clob_token_ids = json.loads(clob_token_ids_str)
        
        if len(clob_token_ids) > 0:
            market_id = clob_token_ids[0]  # First token ID (usually "Yes" outcome)
        else:
            print(f"Skipping {event_title} - no clobTokenIds")
            continue
    else:
        print(f"Skipping {event_title} - no markets")
        continue
    
    # Fetch price history
    url = "https://clob.polymarket.com/prices-history"
    querystring = {"market": str(market_id), "interval": "max", "fidelity": "60"}
    response = requests.get(url, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        
        # Only process if history exists and is not empty
        if "history" in data and len(data["history"]) > 0:
            # Create dataframe for this event
            df = pd.DataFrame(data["history"])
            
            # Convert timestamp to human-readable date
            df["timestamp"] = pd.to_datetime(df["t"], unit="s")
            df["date"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
            df["price"] = df["p"]
            df["event_title"] = event_title
            
            all_data.append(df)
            print(f"✓ {event_title}: {len(df)} price points")
        else:
            print(f"✗ {event_title}: No price history (empty or missing)")
    else:
        print(f"✗ {event_title}: API error {response.status_code}")

# Combine all into one dataframe
if all_data:
    fed_events_ts_df = pd.concat(all_data, ignore_index=True)
    
    # Sort by event title and timestamp
    fed_events_ts_df = fed_events_ts_df.sort_values(["event_title", "timestamp"])
    
    # Select columns to keep
    columns_to_keep = ["date", "timestamp", "event_title", "price"]
    fed_events_ts_df = fed_events_ts_df[columns_to_keep]
    
    print(f"\n=== DATAFRAME CREATED ===")
    print(f"Total rows: {len(fed_events_ts_df)}")
    print(f"Unique events: {fed_events_ts_df['event_title'].nunique()}")
    print(f"Date range: {fed_events_ts_df['timestamp'].min()} to {fed_events_ts_df['timestamp'].max()}")
    print(f"\nFirst few rows:")
    print(fed_events_ts_df.head(10))
    
    # Save to CSV
    fed_events_ts_df.to_csv("fed_events_price_history.csv", index=False)
    print(f"\n✓ Saved to: fed_events_price_history.csv")
else:
    print("⚠ No data collected - no CSV file created")

✓ Fed rate hike in 2025?: 741 price points
✗ How many Fed rate cuts in 2025?: No price history (empty or missing)
✓ Fed emergency rate cut in 2025?: 741 price points
✓ Jerome Powell out as Fed Chair in 2025?: 741 price points
✓ Fed abolished in 2025?: 741 price points
✓ Who will Trump announce as next Fed Chair in 2025?: 741 price points

=== DATAFRAME CREATED ===
Total rows: 3705
Unique events: 5
Date range: 2025-10-06 18:00:03 to 2025-11-06 17:34:07

First few rows:
                     date           timestamp             event_title   price
2223  2025-10-06 18:00:07 2025-10-06 18:00:07  Fed abolished in 2025?  0.0120
2224  2025-10-06 19:00:04 2025-10-06 19:00:04  Fed abolished in 2025?  0.0110
2225  2025-10-06 20:00:05 2025-10-06 20:00:05  Fed abolished in 2025?  0.0130
2226  2025-10-06 21:00:13 2025-10-06 21:00:13  Fed abolished in 2025?  0.0130
2227  2025-10-06 22:00:03 2025-10-06 22:00:03  Fed abolished in 2025?  0.0130
2228  2025-10-06 23:00:10 2025-10-06 23:00:10  Fed abolishe